# Batch processing
This notebook loads table from Postgres, clean it, aggregate and save in `Gold` S3 bucket

### 1. Import libraries

In [ ]:
import pandas as pd
import psycopg2
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col

### 2. Init Spark

In [ ]:
def get_spark():
    builder = SparkSession.builder.appName("BatchETL") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    return configure_spark_with_delta_pip(builder).getOrCreate()

spark = get_spark()
BRONZE_PATH = "/mnt/s3mock/bronze/batch/events"
SILVER_PATH = "/mnt/s3mock/silver/batch/events"
GOLD_PATH = "/mnt/s3mock/gold/batch/aggregates"

## 3. Core logic:

- Reads cleaned data (silver)

- Groups by time and type

- Counts events per group

- Writes aggregated results to a new "gold" Delta table for final consumption (e.g. dashboard, reports, ML)

In [ ]:
conn = psycopg2.connect(
    dbname="airflow",
    user="airflow",
    password="airflow",
    host="postgres"
)
df = pd.read_sql("SELECT * FROM events", conn)
conn.close()

spark_df = spark.createDataFrame(df)
spark_df.write.format("delta").mode("overwrite").save(BRONZE_PATH)

df = spark.read.format("delta").load(BRONZE_PATH)
df.withColumn("event_timestamp", to_date(col("timestamp"))) \
  .write.format("delta").mode("overwrite").save(SILVER_PATH)

df = spark.read.format("delta").load(SILVER_PATH)
agg = df.groupBy("event_timestamp", "event_type") \
        .agg({"user_id": "count", "timestamp": "count"})
agg.write.format("delta").mode("overwrite").save(GOLD_PATH)